In [71]:
import pandas as pd
import numpy as np
import re
import string
import pickle

# Load Model

In [72]:
with open('../static/model/model.pickle', 'rb') as file:
    model = pickle.load(file)

# Load vocabulary

In [73]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header = None)
tokens = vocab[0].tolist()

# Text Preprocessing

In [74]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [75]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [76]:
def preprocessing(text):
    data = pd.DataFrame([text], columns =['tweet'])
    data['tweet'] = data['tweet'].apply(lambda sentence: ' '.join(word.lower() for word in sentence.split()))
    data['tweet'] = data['tweet'].apply(lambda sentence: ' '.join(re.sub(r'^https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE) for word in sentence.split()))
    data['tweet'] = data['tweet'].apply(lambda sentence: re.sub(r'[^\w\s]', '', sentence))
    data['tweet'] = data['tweet'].str.replace('\d+', '', regex = True)
    data['tweet'] = data['tweet'].apply(lambda sentence: ' '.join(word for word in sentence.split() if word not in sw))
    data['tweet'] = data['tweet'].apply(lambda sentence: ' '.join(ps.stem(word) for word in sentence.split()))
    return data['tweet']

# Text Vectorizing

In [77]:
def vectorizer(ds, vocabulary):
    vectorized_lst =[]

    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1

        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype = np.float32)
    return vectorized_lst_new

# Get Prediction

In [98]:
txt = "Great product!. I love it. https://google.com. amazing and i got 4."

In [99]:
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
pred = model.predict(vectorized_txt)
pred

array([0])